In [1]:
from visualizer import calibration
import visual

from skimage.io import imread
from skimage.io.collection import alphanumeric_key
from dask import delayed
from glob import glob
import dask.array as da
import matplotlib.pyplot as plt
import numpy as np
import napari
from napari import Viewer
import os
from PyQt5 import QtCore
from skimage.registration import phase_cross_correlation
import cv2 as cv
from skimage import io

import cupy as cp 
import imagej

%gui qt 
%matplotlib inline
plt.style.use('dark_background')


The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [2]:
loading_path = r'H:\Test\CElegansOH1_5'
stack = calibration.lazy_dask_stack(loading_path,num_cams=25, px_depth='uint16', height=self.M25app.vert, width =self.M25app.horz)

NameError: name 'loading_path' is not defined

In [2]:
# main_folder = r'H:\Test\20220503_M25_200nm'
main_folder = r'H:\Test\20220503_M25_1um_yg'
folder_names = sorted(glob(main_folder + '/CAM*/'), key=alphanumeric_key)
offsets_file = os.path.join(main_folder,"offsets.csv")



# stack= calibration.load_dataset(main_folder,num_cams=25,px_depth='uint8',width=808,height=608)

stack = calibration.load_dataset(main_folder,num_cams=25,px_depth='uint16',width=808,height=608)


In [3]:
#Scope Parameters
# FOV = 50e-6
cam_px =6.0e-6
totalmag = 15.75
px_size_img = cam_px/totalmag
zstep = 2e-6
z_scale = zstep/px_size_img

print(px_size_img)
print(zstep/cam_px)
print(z_scale)

3.8095238095238096e-07
0.3333333333333333
5.25


In [ ]:
# Load offsets from desired folder 
# main_folder = r'E:\Ed\Data\20220324_drosophila_egg\20220324_M25_PSF2-16bit'
main_folder = r'D:\Test\20220428_M25_PSF_200nm_2'

offsets_file_load = os.path.join(main_folder,"offsets.csv")
with open(offsets_file_load) as file_name:
    offsets = np.loadtxt(file_name, delimiter=",")

FileNotFoundError: [Errno 2] No such file or directory: 'D:\\Test\\20220428_M25_PSF_200nm_2\\offsets.csv'

### Get Calibration from Bead's Files

In [4]:
# Generate our own shifted datset
with cp.cuda.Device(0):
    # test = stack[:,11:15,:,:]
    test = stack
    t,c,h,w  = test.shape
    np_mip =[]
    for i in range(c):
        cp_stack = cp.array(test[:,i,:,:])
        cp_mip = cp.max(cp_stack, axis=0)
        np_mip.append(cp.asnumpy(cp_mip))
        cp_mip = None
        cp_stack=None
    np_mip = np.array(np_mip)
    np_mip = np.expand_dims(np_mip,axis=0)
    

In [5]:
# Show the maximum intensity projection
viewer = napari.Viewer()
viewer.add_image(np_mip, name='mip')

<Image layer 'mip' at 0x1f55f5d3580>

In [7]:
#Go to napari and make a square for alignment
crop_region =viewer.layers['Shapes'].data[0]
template = calibration.create_box_ndarray(crop_region)
template_box = template[0:4,2:4]
print(crop_region)
print(template_box)
print(template_box.shape)


[[  0.          12.         122.52152243 371.56263532]
 [  0.          12.         122.52152243 430.42760772]
 [  0.          12.         182.63894106 430.42760772]
 [  0.          12.         182.63894106 371.56263532]]
[[123 372]
 [183 430]]
(2, 2)


In [8]:
# Since OpenCV Match Template takes in uint8 || float32. We can change from uint16 to uint8 easily
np_mip_8bit =visual.im_bit_convert(np_mip,bit=8,norm=True)
min_val, max_val = template_box
t,c = template[0,0:2]
crop_img = np_mip_8bit[t,c, min_val[0]:max_val[0],min_val[1]:max_val[1]]
viewer.add_image(crop_img,name='crop')

<Image layer 'crop' at 0x1f64551dd00>

In [9]:
coord =[]
w,h = template.shape  
methods = 'cv.TM_CCOEFF'
print(w,h)
t,c,h,w = np_mip_8bit.shape

for i in range(c):
    method = eval(methods) # This method parses expression and runs ()
    # Apply template Matching
    # Slide through image and compare template patches.
    # Comparison of best matches is foudn as global minn in SQDIFF or max in CCORR or CCOEF.
    res = cv.matchTemplate(np_mip_8bit[0,i,:,:],crop_img,method)
    min_val, max_val, min_loc, max_loc = cv.minMaxLoc(res)
    # If the method is TM_SQDIFF or TM_SQDIFF_NORMED, take minimum
    if method in [cv.TM_SQDIFF, cv.TM_SQDIFF_NORMED]:
        top_left = min_loc
    else:
        top_left = max_loc
        
    bottom_right = (top_left[0] + w, top_left[1] + h)
    #Output would contain the Top left corner and bottom right in case of need to use this rectangle
    coord.append(((top_left[1],top_left[0]),(bottom_right[1],bottom_right[0])))

#For the bead calibration we only use the TL points for alignment
TL_coords = np.array(coord)
TL_coords= TL_coords[0:25,0]

#Coordinates need to then be subtracted to the reference frame (i.e t,c for which rectangle was drawn)
res = []
ref_cam_index = 0
# diff =[]
for i,coordinate in enumerate(TL_coords):
    val = tuple(map(lambda i, j: i - j, TL_coords[ref_cam_index],coordinate))
    res.append(val)
    #If previous values available we can find teh difference between them
    # diff.append(tuple(map(lambda i, j: i + j, val,shift_stack_coord[i])))
# print(diff)
print(res)
# print(shift_stack_coord)
offset_coordinates = np.array(res)

#Save the Offsets in the folder
np.savetxt(offsets_file, offset_coordinates, delimiter=',')

2 4
[(0, 0), (-35, -33), (-97, -16), (-16, -94), (-29, -68), (72, -14), (12, -11), (28, -49), (-16, -101), (-14, -26), (7, 2), (16, -56), (35, -50), (-3, -63), (27, -45), (13, -46), (20, -54), (34, -62), (-8, -47), (16, -53), (24, -37), (60, -12), (56, -43), (16, -33), (38, -49)]


In [10]:
#Aligning Max Projection to check if coordinates are good

from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset
with cp.cuda.Device(0):
    mip_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(np_mip.shape)
    for i in range(c):
        cp_stack = cp.array(np_mip[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        mip_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    mip_aligned = np.array(mip_aligned)
mip_aligned =  np.moveaxis(mip_aligned,0,1)

#Convert to Dask Array
mip_aligned = da.from_array(mip_aligned)
viewer.add_image(mip_aligned, name='MIP_aligned',scale = [z_scale,1,1])

(25, 3)
(1, 25, 608, 808)


<Image layer 'MIP_aligned' at 0x1f650b2a400>

In [22]:
#Aligning all the stack
from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset
with cp.cuda.Device(0):
    stack_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(stack.shape)
    for i in range(c):
        cp_stack = cp.array(stack[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        stack_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    stack_aligned = np.array(stack_aligned)
stack_aligned =  np.moveaxis(stack_aligned,0,1)

#Convert to Dask Array
stack_aligned = da.from_array(stack_aligned)


(25, 3)
(160, 25, 608, 808)


In [11]:
viewer.add_image(stack_aligned, name='test',scale = [z_scale,1,1])

<Image layer 'test' at 0x17efb8f2ee0>

### Align Datasets with Known Offsets


In [6]:
from cupyx.scipy.ndimage import shift
import cupy as cp
# Generate our own shifted datset

offset_coordinates = offsets
with cp.cuda.Device(0):
    stack_aligned = []
    t,c,h,w = stack.shape
    shift_stack_cam = cp.zeros((c,h,w))
    columns = cp.zeros((offset_coordinates.shape[0],1))
    shift_stack_coord = cp.hstack((columns,offset_coordinates))
    print(shift_stack_coord.shape)
    print(stack.shape)
    for i in range(c):
        cp_stack = cp.array(stack[:,i,:,:])
        shift_stack_cam= shift(cp_stack,shift_stack_coord[i])
        stack_aligned.append(cp.asnumpy(shift_stack_cam))
        cp_stack = None
    stack_aligned = np.array(stack_aligned)
stack_aligned =  np.moveaxis(stack_aligned,0,1)

(24, 3)
(42, 25, 608, 808)


IndexError: Index 24 is out of bounds for axis 0 with size 24

In [ ]:
#Convert numpy to dask?
da_stack_align = da.from_array(stack_aligned)
print(da_stack_align)

dask.array<array, shape=(46, 24, 608, 808), dtype=uint8, chunksize=(46, 24, 304, 202), chunktype=numpy.ndarray>


In [7]:
viewer= napari.Viewer()

In [10]:
viewer.add_image(da_stack_align, name='dask_align',scale=[z_scale,1,1])

NameError: name 'z_scale' is not defined

### Image J Part

In [23]:

def start_imagej():
    import imagej
    global ij
    ij = imagej.init( mode='interactive')
    # path = r'C:\Users\yoshi\Documents\fiji-win64\Fiji.app'
    # ij = imagej.init(path,mode='interactive')
    ij.ui().showUI()
    print(ij.getVersion())

QtCore.QTimer.singleShot(0, start_imagej)

In [8]:
# def start_imagej():
#     global ij
#     ij = imagej.init(headless=False)
#     ij.ui().showUI()
#     print(ij.getVersion())

# QtCore.QTimer.singleShot(0, start_imagej)

In [1]:
# viewer= napari.Viewer()

NameError: name 'napari' is not defined

In [9]:
# # Align the files
# ij_stack2  = ij.py.to_java(np.array(stack,dtype='uint8'))
# ij.ui().show('aligned_stack', ij_stack2)

In [25]:
# Align the files
ij_stack2  = ij.py.to_java(np.array(stack_aligned,dtype='uint16'))
ij.ui().show('aligned_stack', ij_stack2)

In [ ]:
                aligned_volume_cp.shape
cuda_mem_info()
cuda_clear()